# Witcher Network Analysis

This notebook recreates the analysis from Chapter 1 of "Data Science from Scratch" by Joel Grus, applying it to the Witcher universe. Instead of analyzing friendships, we analyze character interactions across scenes in the books.

## Import Required Libraries

Load pandas for data manipulation and analysis.

In [ ]:
import pandas as pd

## Load Data

Read the Witcher network dataset from CSV file.

In [ ]:
df_interactions = pd.read_csv('witcher_network.csv')
df_interactions.head()

## Clean Data

Remove the unnamed index column created during CSV export.

In [ ]:
unnamed_col = df_interactions.columns[0]
df_interactions.drop(unnamed_col, axis=1, inplace=True)
df_interactions.head()

## Filter Book 1

Extract interactions only from the first book.

In [ ]:
df_book1 = df_interactions[df_interactions['book'] == 1]
df_book1

## Extract Unique Characters

Get all unique characters that appear in Book 1.

In [ ]:
all_characters = pd.concat([df_book1['Source'],df_book1['Target']]).unique()
all_characters

## Create Character List

Build a list of character dictionaries with their unique IDs and names.

In [ ]:
characters = [{"id": i, "name": character}
              for i, character in enumerate(all_characters)]
characters

## Create Name-to-ID Mapping

Build a dictionary for quick lookup of character IDs by name.

In [ ]:
character_name_to_id = {character["name"]:character["id"]
                        for character in characters}
character_name_to_id

## Convert Interactions to ID Pairs

Transform character name interactions into tuples of character IDs.

In [ ]:
interaction_pairs_by_name = list(zip(df_book1['Source'],df_book1['Target']))
interaction_pairs_by_id = [(character_name_to_id[source], character_name_to_id[target]) 
                           for source, target in interaction_pairs_by_name]
interaction_pairs_by_id

## Initialize Friendships Graph

Create a dictionary to store the interaction network for each character.

In [ ]:
character_interactions = {character["id"]: [] for character in characters}
character_interactions

## Build Interaction Network

Populate the friendships dictionary with bidirectional interactions.

In [ ]:
for source_id, target_id in interaction_pairs_by_id:
    character_interactions[source_id].append(target_id)
    character_interactions[target_id].append(source_id)
character_interactions

## Calculate Interaction Count

Define a function to count the number of characters each character interacts with.

In [ ]:
def number_of_interactions(character):
    """Return the total number of interactions for a given character."""
    return len(character_interactions[character["id"]])
number_of_interactions(characters[1])

## Sort Characters by Interaction Count

Create a sorted list of characters ranked by their number of interactions.

In [ ]:
interaction_counts = [(character["id"], number_of_interactions(character))
                      for character in characters]
interaction_counts_sorted = sorted(interaction_counts, key=lambda x: x[1], reverse=True)
interaction_counts_sorted

## Display Top 5 Most Connected Characters

Show the names of the five characters with the most interactions.

In [ ]:
for i in range(0, 5):
    print(characters[interaction_counts_sorted[i][0]]["name"])

## Friends of Friends Analysis

Define a function that identifies potential connection opportunities by finding characters who share mutual interactions.

In [ ]:
from collections import Counter

def friends_of_friends(character):
    """
    Return a Counter of characters connected through mutual interactions.
    Keys are character IDs, values are the count of shared connections.
    """
    mutual_connections = Counter()
    for direct_connection in character_interactions[character["id"]]:
        for mutual_connection in character_interactions[direct_connection]:
            if (mutual_connection != character["id"] and 
                mutual_connection not in character_interactions[character["id"]]):
                mutual_connections[mutual_connection] += 1
    return mutual_connections

## Display Friends of Friends

Show the ordered list of potential connections for the first character, ranked by number of shared interactions.

In [ ]:
print(friends_of_friends(characters[0]).most_common())